# Dealing with missing values
We are going to work on three approaches to dealing with the missing values in a dataset. These are: <br>
1. Dropping Columns with Missing Values
2. Imputation (basically we're just filling the empty value, for example we can fill in the mean value along each column)
3. An Extension To Imputation (but the value we delete was special for the dataset, so we're adding a new column that shows the location of the imputed entries) <br>

## Here is how we use it: 

In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

melb_data = pd.read_csv('./melb_data.csv')

# what we want to predict
y = melb_data.Price

predictors = melb_data.drop(["Price"], axis=1)
X = predictors.select_dtypes(exclude="object")

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=1)

def score_dataset(train_X, valid_X, train_y, valid_y):
    model = RandomForestRegressor(n_estimators=10, random_state=1)
    model.fit(train_X, train_y)
    y_pred = model.predict(valid_X)
    mae = mean_absolute_error(valid_y, y_pred)
    return mae


In [3]:
# Shape of training data (num_rows, num_columns)
print(train_X.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (train_X.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])


(10864, 12)
Car               52
BuildingArea    5193
YearBuilt       4312
dtype: int64


### 1st Approach: Dropping Columns

In [4]:
missing_columns = [col for col in train_X.columns if train_X[col].isnull().any()]
reduced_X_train = train_X.drop(missing_columns, axis=1)
reduced_X_valid = valid_X.drop(missing_columns, axis=1)

### 2nd Approach: Imputation

In [5]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(train_X))
imputed_X_valid = pd.DataFrame(my_imputer.transform(valid_X))

# Imputation removed column names; put them back
imputed_X_train.columns = train_X.columns
imputed_X_valid.columns = valid_X.columns

### 3rd Approach

In [6]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = train_X.copy()
X_valid_plus = valid_X.copy()

# Make new columns indicating what will be imputed
for col in missing_columns:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns


1041     False
1989     False
10157    False
1711     False
11565    False
         ...  
905      False
5192     False
12172    False
235      False
13349    False
Name: Car_was_missing, Length: 10864, dtype: bool
1041     False
1989     False
10157     True
1711     False
11565     True
         ...  
905      False
5192      True
12172    False
235       True
13349     True
Name: BuildingArea_was_missing, Length: 10864, dtype: bool
1041     False
1989     False
10157     True
1711     False
11565     True
         ...  
905      False
5192      True
12172    False
235       True
13349     True
Name: YearBuilt_was_missing, Length: 10864, dtype: bool


### Comparison

In [7]:
print("MAE from Approach 1:")
print(score_dataset(reduced_X_train, reduced_X_valid, train_y, valid_y))
print("MAE from Approach 2:")
print(score_dataset(imputed_X_train, imputed_X_valid, train_y, valid_y))
print("MAE from Approach 3:")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus,  train_y, valid_y))


MAE from Approach 1:
187082.57548478153
MAE from Approach 2:
174917.69150711832
MAE from Approach 3:
177346.30320324007


So, approach 2 better for our dataset. The training data has 10864 rows and 12 columns, where three columns contain missing data. For each column, less than half of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

In [8]:
# Shape of training data (num_rows, num_columns)
print(reduced_X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (train_X.isnull().sum())
print(train_X.isnull().any().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0.0])
print("total: ", train_X.isnull().sum().sum())


(10864, 9)
3
Car               52
BuildingArea    5193
YearBuilt       4312
dtype: int64
total:  9557
